In [1]:
import os
import torch
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from tqdm.auto import tqdm
import warnings

# 🔕 Disattiva warning su torch.load
warnings.filterwarnings("ignore", category=FutureWarning)

# 📁 Percorsi
root_input = "/content/drive/MyDrive/NeuroOnco/Derivate"
root_output = "/content/drive/MyDrive/NeuroOnco/Preprocessate"
os.makedirs(root_output, exist_ok=True)

# 🔍 Lista dei file .pt
file_pt = sorted([f for f in os.listdir(root_input) if f.endswith(".pt")])

# 🔘 UI: selezione multipla + bottone
multi_select = widgets.SelectMultiple(
    options=file_pt,
    description='File:',
    layout=widgets.Layout(width='60%', height='200px')
)
run_button = widgets.Button(description="Esegui preprocessing", button_style='success')
display(widgets.VBox([multi_select, run_button]))


# ⚙️ Funzione di normalizzazione
def normalizza_volume(volume, nan_value=-1000.0):
    volume = volume.copy()
    for c in range(volume.shape[0]):
        print(f"🔄 Normalizzo canale {c+1}/{volume.shape[0]}")
        vol_c = volume[c]
        valid = ~np.isnan(vol_c)
        if not np.any(valid):
            print(f"⚠️ Canale {c+1} tutto NaN, salto.")
            continue
        sample = vol_c[valid]
        if sample.size > 1_000_000:
            sample = np.random.choice(sample, 1_000_000, replace=False)
        p1, p99 = np.percentile(sample, [1, 99])
        vol_c = (vol_c - p1) / (p99 - p1 + 1e-5)
        vol_c = np.clip(vol_c, 0, 1)
        vol_c[~valid] = np.nan
        volume[c] = vol_c

    # ✅ sostituzione canale per canale
    for c in range(volume.shape[0]):
        n_nan = np.isnan(volume[c]).sum()
        if n_nan > 0:
            print(f"🧹 Canale {c+1}: sostituisco {n_nan} NaN con {nan_value}")
            volume[c][np.isnan(volume[c])] = nan_value

    return volume


# 🔁 Funzione di elaborazione
def esegui_preprocessing(file_selezionati):
    print(f"\n📋 Inizio preprocessing di {len(file_selezionati)} file...")
    for fname in tqdm(file_selezionati, desc="Elaborazione", unit="file"):
        path_in = os.path.join(root_input, fname)
        path_out = os.path.join(root_output, fname)
        errore = None

        try:
            print(f"\n🔍 Caricamento: {fname}")
            data = torch.load(path_in, weights_only=False)

            if "volume" not in data:
                raise ValueError("Il file non contiene 'volume'")

            volume_torch = data["volume"]
            if not isinstance(volume_torch, torch.Tensor):
                raise TypeError("Il campo 'volume' non è un tensore")

            print(f"📐 Volume shape: {volume_torch.shape}, dtype: {volume_torch.dtype}")
            volume = volume_torch.to(dtype=torch.float32).numpy()

            print("⚙️ Normalizzazione in corso...")
            volume_proc = normalizza_volume(volume)

            data_proc = {
                "volume": torch.tensor(volume_proc, dtype=torch.float16),
                "roi_masks": data.get("roi_masks", {}),
                "nomi_serie": data.get("nomi_serie", [])
            }

            torch.save(data_proc, path_out)
            print(f"✅ {fname} salvato in Preprocessate.")

        except Exception as e:
            errore = str(e)
            print(f"❌ Errore con {fname}: {errore}")


# 🟢 Callback del bottone
def on_run_button_clicked(b):
    selezionati = list(multi_select.value)
    if not selezionati:
        print("⚠️ Nessun file selezionato.")
        return
    esegui_preprocessing(selezionati)

run_button.on_click(on_run_button_clicked)



📋 Inizio preprocessing di 6 file...


Elaborazione:   0%|          | 0/6 [00:00<?, ?file/s]


🔍 Caricamento: ID1.pt
📐 Volume shape: torch.Size([6, 340, 480, 480]), dtype: torch.float16
⚙️ Normalizzazione in corso...
🔄 Normalizzo canale 1/6
🔄 Normalizzo canale 2/6
🔄 Normalizzo canale 3/6
🔄 Normalizzo canale 4/6
🔄 Normalizzo canale 5/6
🔄 Normalizzo canale 6/6
✅ ID1.pt salvato in Preprocessate.

🔍 Caricamento: ID2.pt
📐 Volume shape: torch.Size([6, 340, 480, 480]), dtype: torch.float16
⚙️ Normalizzazione in corso...
🔄 Normalizzo canale 1/6
🔄 Normalizzo canale 2/6
🔄 Normalizzo canale 3/6
🔄 Normalizzo canale 4/6
🔄 Normalizzo canale 5/6
🔄 Normalizzo canale 6/6
✅ ID2.pt salvato in Preprocessate.

🔍 Caricamento: ID3.pt
📐 Volume shape: torch.Size([6, 340, 560, 560]), dtype: torch.float16
⚙️ Normalizzazione in corso...
